In [ ]:
import org.apache.spark.sql.{SparkSession, DataFrame}
import org.apache.spark.sql.functions._

In [1]:
val spark = SparkSession.builder()
      .appName("Broadcast Join Example")
      .master("local[*]") // Use local mode for simplicity
      .getOrCreate()
val sparkContext = spark.sparkContext

spark = org.apache.spark.sql.SparkSession@1176d7a0
sparkContext = org.apache.spark.SparkContext@6e19d0c0


org.apache.spark.SparkContext@6e19d0c0

In [2]:
val transactionsLocation = "gs://artifacts_spark_jobs/movie_metadata/yaseen/transaction_logs.csv"
val usersData = "gs://artifacts_spark_jobs/movie_metadata/yaseen/user_details.csv"

transactionsLocation = gs://artifacts_spark_jobs/movie_metadata/yaseen/transaction_logs.csv
usersData = gs://artifacts_spark_jobs/movie_metadata/yaseen/user_details.csv


gs://artifacts_spark_jobs/movie_metadata/yaseen/user_details.csv

In [3]:
val transactionsData = spark.read.format("csv").option("header","true").option("inferSchema","true").load(transactionsLocation)

transactionsData = [user_id: int, transaction_type: string ... 1 more field]


[user_id: int, transaction_type: string ... 1 more field]

In [4]:
transactionsData.show()

+-------+----------------+------+
|user_id|transaction_type|amount|
+-------+----------------+------+
|    623|          Refund|357.42|
|    393|        Purchase|518.77|
|    702|        Purchase|980.64|
|    829|        Purchase|107.04|
|     39|        Purchase|736.92|
|    394|          Refund|660.84|
|    376|        Purchase|782.27|
|    125|          Refund| 438.5|
|    257|          Refund|614.49|
|    966|        Purchase|986.76|
|    278|        Purchase|506.34|
|    918|        Purchase|903.83|
|    462|        Purchase| 50.09|
|    581|        Purchase|744.69|
|    948|        Purchase|224.01|
|    612|          Refund|588.98|
|    598|        Purchase|700.02|
|    836|          Refund|549.84|
|    682|          Refund|881.29|
|    733|          Refund|943.92|
+-------+----------------+------+
only showing top 20 rows



In [5]:
val userData = spark.read.format("csv").option("header", "true").option("inferSchema","true").load(usersData)

userData = [user_id: int, name: string]


[user_id: int, name: string]

In [6]:
userData.show()

+-------+-------+
|user_id|   name|
+-------+-------+
|      1| User_1|
|      2| User_2|
|      3| User_3|
|      4| User_4|
|      5| User_5|
|      6| User_6|
|      7| User_7|
|      8| User_8|
|      9| User_9|
|     10|User_10|
|     11|User_11|
|     12|User_12|
|     13|User_13|
|     14|User_14|
|     15|User_15|
|     16|User_16|
|     17|User_17|
|     18|User_18|
|     19|User_19|
|     20|User_20|
+-------+-------+
only showing top 20 rows



In [9]:
val userDF = userData.toDF

userDF = [user_id: int, name: string]


[user_id: int, name: string]

In [10]:
val transactionsDF = transactionsData.toDF

transactionsDF = [user_id: int, transaction_type: string ... 1 more field]


[user_id: int, transaction_type: string ... 1 more field]

In [12]:
import org.apache.spark.sql.functions.broadcast 
val joinedData = transactionsDF.join(
      broadcast(userDF), // Broadcast the smaller dataset
      Seq("user_id"),
      "inner" // Perform an inner join
    )

joinedData = [user_id: int, transaction_type: string ... 2 more fields]


[user_id: int, transaction_type: string ... 2 more fields]

In [13]:
joinedData.show

+-------+----------------+------+--------+
|user_id|transaction_type|amount|    name|
+-------+----------------+------+--------+
|    623|          Refund|357.42|User_623|
|    393|        Purchase|518.77|User_393|
|    702|        Purchase|980.64|User_702|
|    829|        Purchase|107.04|User_829|
|     39|        Purchase|736.92| User_39|
|    394|          Refund|660.84|User_394|
|    376|        Purchase|782.27|User_376|
|    125|          Refund| 438.5|User_125|
|    257|          Refund|614.49|User_257|
|    966|        Purchase|986.76|User_966|
|    278|        Purchase|506.34|User_278|
|    918|        Purchase|903.83|User_918|
|    462|        Purchase| 50.09|User_462|
|    581|        Purchase|744.69|User_581|
|    948|        Purchase|224.01|User_948|
|    612|          Refund|588.98|User_612|
|    598|        Purchase|700.02|User_598|
|    836|          Refund|549.84|User_836|
|    682|          Refund|881.29|User_682|
|    733|          Refund|943.92|User_733|
+-------+--

In [14]:
spark.stop

In [ ]:
// Without Broadcasting: A standard join in Spark requires shuffling, where data from both datasets is exchanged across the network to 
// co-locate matching keys. This process involves moving large amounts of data between worker nodes, 
// which is both time-consuming and resource-intensive.


// With Broadcasting: The small dataset is sent to all worker nodes. 
// Each worker can perform the join locally, eliminating the need for shuffling the large dataset.